<a href="https://colab.research.google.com/github/VKapicyn/m06-006sk-course-work-var2/blob/master/Ant_Colony_on_1024Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Эксперименты с графом

In [2]:
import networkx as nx

In [3]:
G = nx.gnm_random_graph(1024, 6000, directed=True)

In [ ]:
sum = 0
zeros = 0
for i in G:
  sum += G.degree[i]
  if G.degree[i] == 0:
    zeros += 1
print(sum/G.number_of_nodes())
print(zeros)

11.71875
0


In [ ]:
G.number_of_nodes()

1024

In [ ]:
print(nx.has_path(G,1,500))

True


In [ ]:
a = set()
for i in range(1,1024): 
  a.add(nx.shortest_path_length(G,0,i))
print(min(a), max(a))

1 5


In [ ]:
k = 0
for i in range(0,1024):
  if nx.has_path(G,6,i):
    k += 1
print(k)

1024


# Эксперименты с муравьями

Следующий блок кода является модифицированным кодом из репозитория https://github.com/Akavall/AntColonyOptimization

---


Этот блок кода реализует прохождение муравьиной колонии по графу с целью поиска кратчайшего пути (ещё в доработке)

Алгоритм состоит из N итераций. Кол-во итераций задаётся в параметр n_iterations.

---
Содержание каждой итерации:

1. Выпускается K муравьёв (задаётся параметром n_ants) в функции **gen_all_paths()**
2. Собираются путь каждого муравья (ф-я **gen_path()**) при том, что муравей выбирает, куда ему идти в функции **pick_move()**
3. Для каждого собранного пути считается его дистанция (ф-я **gen_path_dist()**)
4. В зависимости от длинны дистанции по неё распыляется феромон (ф-я **spread_pheromone()**)




In [ ]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):

  def __init__(self, graph, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):

    self.graph = graph
    self.distances  = nx.to_numpy_array(graph)
    self.pheromone = np.ones(self.distances.shape) / len(self.distances)
    self.n_ants = n_ants
    self.n_best = n_best
    self.n_iterations = n_iterations
    self.decay = decay
    self.alpha = alpha
    self.beta = beta

  #ГОТОВО
  def run(self):
    shortest_path = None
    all_time_shortest_path = ("placeholder", np.inf)
    for i in range(self.n_iterations):
        all_paths = self.gen_all_paths() #получаем все пути муравьёв за эту итерацию
        self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path) #распыляем феромоны на путях в зависимости от их итоговой длинны
        shortest_path = min(all_paths, key=lambda x: x[1]) #находим кратчайший путь из всех муравьиных
        print (shortest_path)
        #если нашли более короткий путь, чем был раньше, то заменяем
        if shortest_path[1] < all_time_shortest_path[1]:
            all_time_shortest_path = shortest_path            
        self.pheromone * self.decay #испаряем феромон
    return all_time_shortest_path

  #ГОТОВО
  def spread_pheronome(self, all_paths, n_best, shortest_path):
    sorted_paths = sorted(all_paths, key=lambda x: x[1]) #сортируем пути по дистанции от минимальной к максимальной
    for path, dist in sorted_paths[:n_best]: #распыляем феромон на пути с наименьшей дистанцией
        for move in path:
            self.pheromone[move] += 1.0 / dist #self.distances[move] #к феромону прибавляется значение обратно пропорциональное пройденной дистанции
  
  #ГОТОВО: суммируем длинну пути
  def gen_path_dist(self, path):
    total_dist = 0
    for step in path:
        total_dist += self.distances[step]
    return total_dist

  #ГОТОВО: собираем пути всех муравьёв
  def gen_all_paths(self):
    all_paths = []
    # выпускаем N муравьёв
    for i in range(self.n_ants):
        path = self.gen_path(0) # каждый муравей проходит свой путь, начиная с 0-го узла
        all_paths.append((path, self.gen_path_dist(path))) # этот путь присоединяется к копилке всех путей этой итерации муравейника
    return all_paths

  # то, как муравей проходит свой путь
  def gen_path(self, start):
    path = []
    #visited = set()
    #visited.add(start)
    prev = start
    i = 0
    while (prev != 1023): # цикл до попадания муравья в нужный узел
      move = self.pick_move(self, self.pheromone, prev) # муравей выбирает следующий шаг
      path.append((prev,move))
      # муравей, зашедший в тупик, возвращается на базу до следующего выхода
      if G.degree[move] == 1:
        break
      # муравей, гуляющий слишком долго, возвращается на базу до следующего выхода
      i += 1
      if i >= 3000:
        break
      prev = move
    return path
  

  def pick_move(self, pheromone, dist, prev):
      pheromone = np.copy(pheromone)
      neighbours = list(self.graph.adj[prev]) # список соседей (вариантов)
      phers = []
      for i in neighbours:
        phers.append(pheromone[i][prev]) # список феромонов на соседях

      row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta) # предпочтительность соседа при выборе пути

      norm_row = row / row.sum() # нормированная предпочтительность
      move = np_choice(neighbours, 1, p=norm_row)[0] # волшебство выбора
      return move

In [ ]:
G.graph

{}

In [ ]:
ants = AntColony(G, 20, 5, 100, 0.95, alpha=1, beta=1)
shortest_path = ants.run()

IndexError: ignored